# Data Preparation


**In this notebook we will prepare our data for our search function to useCurrently we have data stored in two different csv files**


.rimi main.csv

.Rimi1.csv

**It can be computationally expensive to produce analysis results from multiple data-sources for incomming stream of requests.
So we will prepare our data and save it in an easily searchable structure**

In [1]:
# Import the needed modules...
import pandas as pd
from collections import defaultdict
from os import getcwd

# Data Engineering
.##Get data in dataframes.

.##Convert data to a single dictionary.

In [2]:
"""
    Read data from Rimi1.csv
"""
Rimi= pd.read_csv('Rimi1.csv')
Rimi_table_columns = Rimi.columns.tolist()
print(f"COLUMNS : {Rimi_table_columns}")

COLUMNS : ['Pid', 'mrp', 'sale_price', 'discount_percentage', 'seller_name']


In [3]:
"""
    Read data from rimi main.csv
"""
kajal= pd.read_csv('rimi main.csv')
kajal_table_columns = kajal.columns.tolist()
print(f"COLUMNS : {kajal_table_columns}")

COLUMNS : ['product_name', 'Pid', 'product_url', 'brand_name', 'image_url', 'product_description', 'date_first_available']


.### Pid is a common column in all two tables so we will use it as a primary search-key

.### Pid is a common key across two tables, so we will use it as a sort key...

.### A user will always search a movie by its Pid so we will create a Global secondary index to be able to perform search our datastore it will obviously take some extra space but almost negligible as compared to the size of the original data.In addition, It will make our searching faster and efficient so it's a good deal.

In [4]:
print(f"It is {pd.Series(Rimi['Pid']).is_unique}  that the column 'Pid' has unique values for all entries in Rimi1 dataframe.")
print(f"It is {pd.Series(kajal['Pid']).is_unique}  that the column 'Pid' has unique values for all entries in rimi main dataframe.")
# Sort Rimi1 dataframe on the basis of Pid as Pid is unique for all entries...
Rimi_sorted = Rimi.sort_values(by=['Pid'])

# Sort rimi main dataframe on the basis of Pid as Pid is unique for all entries...
kajal_sorted  = kajal.sort_values(by=['Pid'])

It is True  that the column 'Pid' has unique values for all entries in Rimi1 dataframe.
It is True  that the column 'Pid' has unique values for all entries in rimi main dataframe.


In [5]:

# from Rimi1 dataframe...
Pids    = kajal_sorted["Pid"].tolist()
product_url = kajal_sorted["product_url"].tolist()
product_name=kajal_sorted["product_name"].tolist()
brand_name=kajal_sorted["brand_name"].tolist()
image_url=kajal_sorted["image_url"].tolist()
product_disc=kajal_sorted["product_description"].tolist()
date_first=kajal_sorted["date_first_available"].tolist()





# from rimi main dataframe...
MRP=Rimi_sorted["mrp"].tolist()
sale_price=Rimi_sorted["sale_price"].tolist()
discount_price=Rimi_sorted["discount_percentage"].tolist()
seller_name=Rimi_sorted["seller_name"].tolist()



In [6]:
kajalDict             = {}
global_secondaryIndex = {}
kajallist=[]
for idx, Pid in enumerate(Pids):
    kajallist.append(Pid)
    kajalDict[Pid] ={
        
        "producturl" : product_url[idx],
        "productname" : product_name[idx],
        "imageurl" : image_url[idx],
        "brandname" : brand_name[idx],
        "productdicrip":product_disc[idx],
        "datefirst" : date_first[idx],
        
        "rimi" : {
            "mrp" : MRP[idx], 
            "saleprice" : sale_price[idx],
            "discountprice" : discount_price[idx],
            "sellername": seller_name[idx]
            
        }
    }
    
    global_secondaryIndex[Pid] = product_url[idx]

In [7]:
# delete veriables which are no longer in use while holding large amount of data.
del Pids
del product_url
del product_name
del brand_name
del image_url
del product_disc
del date_first
del MRP
del seller_name
del sale_price
del discount_price

In [8]:
import json
print("[INFO] Writing Rimi1 data into the disk...")
with open('kajal.json', 'w') as fp:
    json.dump(kajalDict, fp, sort_keys=True, indent=4)
print("[INFO] Writing Global Secondary Index Data into the disk...")
with open('dataFinal_GIS.json', 'w') as fp:
    json.dump(global_secondaryIndex, fp, sort_keys=True, indent=4)

[INFO] Writing Rimi1 data into the disk...
[INFO] Writing Global Secondary Index Data into the disk...


***At this point, our database is ready and it can handel high inflow of requests***